## Predict what color a card is based on it's text in Magic the Gathering

# Install Wizards of the Coast's API wrapper for python

In [ ]:
!pip install mtgsdk

# Import nessesary tools

In [ ]:
import numpy as np
import pandas as pd
from mtgsdk import Card
from mtgsdk import Set

# Collect Data

In [ ]:
def collect_magic_set(mtg_set_code="###"):
    cards = Card.where(set=mtg_set_code).all()
    return cards

def get_text(magic_set):
    """Uses wizards API to build a dictionary of cards for one "magic_set": name, text, and colors"""
    whole_set = []
    for card in magic_set:
        cards = {
            'text' : card.text,
            'colors' : card.colors,
        }
        
        whole_set.append(cards)
    return whole_set

def get_df(mtg_set_alias):
    magic_set = collect_magic_set(mtg_set_alias)
    magic_set_json = get_text(magic_set)
    magic_set_df = pd.DataFrame(magic_set_json)
    return magic_set_df

In [ ]:
m10_df = get_df('m10')

In [ ]:
# Preprocessing

# vanilla creatures are creatures with no text and just stats and a name
m10_df['vanilla_creature'] = m10_df['text'].apply(lambda x : 1 if x is None else 0)

In [ ]:
m10_df = m10_df.replace('\n',' ', regex=True)
m10_df

In [ ]:
m13_df = get_df('m13')
m15_df = get_df('m15')
m16_df = get_df('m16')
m17_df = get_df('m17')
m20_df = get_df('m20')